In [159]:
%cd ~/advent_of_code_2024
from utils.get_input import get_input
data = get_input(12).strip()
data = data.splitlines()

/home/arseny/advent_of_code_2024


In [160]:
visited, regions = set(), []
directions = [(1,0),(0,1),(-1,0),(0,-1)]

def search(i,j,region):
    region.add((i,j))
    visited.add((i,j))
    r = data[i][j]
    for d in directions:
        ni, nj = i+d[0], j+d[1]
        if 0 <= ni < len(data) and 0 <= nj < len(data[0]) and (ni,nj) not in visited: 
            if data[ni][nj] == r:
                search(ni,nj,region)

for i,row in enumerate(data):
    for j,c in enumerate(row):
        if (i,j) not in visited:
            p = set()
            search(i,j,p)
            regions.append(p)

In [161]:
total = 0
for region in regions:
    area = len(region)
    perimeter = 0
    for plot in region:
        i,j = plot[0],plot[1]
        for d in directions:
            ni, nj = i+d[0], j+d[1]
            if (ni,nj) not in region:
                perimeter += 1
    total += area*perimeter
total

1374934

In [165]:
edge_transforms = {(1,0):((1,0),(1,1)),
                  (0,1):((1,1),(0,1)),
                  (-1,0):((0,1),(0,0)),
                  (0,-1):((0,0),(1,0))}

def subtract_tuples(p1,p2):
    return (p2[0]-p1[0],p2[1]-p1[1])

region_edges = []
for region in regions:
    area, edges, p2s = len(region), {}, set()
    for plot in region:
        i,j = plot[0],plot[1]
        for d in directions:
            ni, nj = i+d[0], j+d[1]
            if (ni,nj) not in region:
                et = edge_transforms[d]
                p1, p2 = (i+et[0][0],j+et[0][1]), (i+et[1][0],j+et[1][1])
                
                # handle moebius edge case
                if p1 in edges: p1 = (p1[0],p1[1],1)
                elif p2 in p2s: p2 = (p2[0],p2[1],1)
                p2s.add(p2)
        
                edges[p1] = p2
        
    region_edges.append((area,edges))

def find_corner(edges):
    p1 = list(edges.keys())[0]
    p2 = edges[p1]
    d1 = subtract_tuples(p1,p2)
    d2 = d1
    while d1 == d2:
        p1,d1 = p2,d2
        p2 = edges[p1]
        d2 = subtract_tuples(p1,p2)
    return (p1,d2)

total = 0
for region in region_edges:
    area, edges = region
    visited, sides = set(), 0
    while len(edges) > 0:
        p1, d1 = find_corner(edges)
        while p1 not in visited:
            visited.add(p1)
            p2 = edges[p1]
            d2 = subtract_tuples(p1,p2)
            if d1 != d2:
                sides += 1
                d1 = d2
            del edges[p1]
            p1 = p2
        sides += 1

    total += area*sides
    
total 

841078